# Particle Filter for Object Tracking

The particle filter is a general algorithm capable of addressing various problem types,
with a particular strength in solving estimation problems. It excels in estimating the
states of systems with _multimodal_ states, a task that conventional Kalman filters,
including the _classic Kalman filter_, _extended Kalman filter_, and _unscented Kalman
filter_, struggle to handle effectively. In this tutorial, I will explore the
application of the particle filter for object tracking through two illustrative
examples. The first example can be addressed using algorithms from the Kalman filter
family, whereas the second problem demands the unique capabilities of the particle
filter.

**RULES:** As usual, **`OpenCV`** is banned in this repository.

Please note that this tutorial will adhere to the notation used in the
[Particle Filter's Wikipedia page](https://en.wikipedia.org/wiki/Particle_filter),
ensuring that readers can easily refer to the source for additional information when
needed.


In [ ]:
from typing import List, Tuple
import cv2
import time
import imageio
import numpy as np
import matplotlib.pyplot as plt
# from skimage import io, color
from particle_filter_utils import *
from functools import partial

video_reader = imageio.get_reader("./input/pres_debate.avi")
frames = [np.array(frame) for frame in video_reader]
video_reader.close()
president_video = np.array(frames)
assert president_video.ndim == 4  # n x H x W x 3

video_reader = imageio.get_reader("./input/pedestrians.avi")
frames = [np.array(frame) for frame in video_reader]
video_reader.close()
blonde_video = np.array(frames)
assert blonde_video.ndim == 4  # n x H x W x 3

## Introduction

This tutorial focuses on the application of a particle filter for tracking objects over
time. It's essential to clarify that we are specifically addressing a tracking problem,
not a detection problem. In other words, our objective is not to detect an object but to
track it once it's already known to us. To illustrate this, consider the scenario where
we have initial information about a car's position, and our goal is to autonomously
monitor and update the car's position continuously throughout a given time period.

When tracking an object, the primary objective is to deduce a sequence of world states,
denoted as $x_k$, from a noisy sequence of measurements or observations, denoted as
$y_k$. Particle filters share similarities with Kalman filters, as they consist of two
essential components: the dynamic or temporal model, denoted as $g(\cdot)$, and the
measurement model, denoted as $h(\cdot)$.

-   The dynamic or temporal model, $g(\cdot)$, characterizes the relationship between
    successive states. Typically, particle filters make use of the Markov assumption,
    which posits that each state depends solely on its predecessor, represented as
    $P(x_k | x_{k-1})$.

-   The measurement model, $h(\cdot)$, describes the connection between the measurement
    $y_k$ and the state $x_k$ at time $k$. We consider this model as generative, and it
    helps us model the likelihood, $P(y_k | x_k)$.

By leveraging this statistical dependency, we can infer the state, $x_k$, even when the
associated observation, $y_k$, provides partial or no informative content.

In the context of inference, the primary challenge is to calculate the marginal
posterior distribution:

$$
P(x_k|y_{0\dots k}) = \frac{P(y_k|x_k)P(x_k|y_{0\dots {k-1}})}{\int P(y_k|x_k)
P(x_k|y_{0 \dots {k-1}}) dx}
$$

To evaluate $P(x_k | y_{0\dots k})$, we need to determine $P(x_k | y_{0\dots {k-1}})$,
which signifies our prior knowledge about the state $x_k$ before incorporating the
associated measurement $y_k$. This can be computed as follows:

$$
P(x_k|y_{0 \dots k-1}) = \int P(x_k|x_{k-1}) P(x_{k-1}|y_{0 \dots {k-1}}) dx_{k-1}
$$

One of the simplest particle filter methods is the _conditional density propagation_ or
_condensation_ algorithm. In this algorithm, the probability distribution
$P(x_k|y_{0\dots k-1})$ is represented by a weighted sum of particles. The following
intuitive image provides an overview of how the condensation algorithm works:

a) The posterior at the previous step is represented as a set of weighted particles.

b) The particles are resampled according to their weights to produce a new set of
unweighted particles.

c) These particles are passed through the nonlinear temporal function.

d) Noise is added according to the temporal model.

e) The particles are passed through the measurement model and compared to the
measurement density.

f) The particles are re-weighted according to their compatibility with the measurements,
and the process can begin again.

<img src="./images/Particle.svg" alt="Alternative Text">

The image and the accompanying description provided above have been adapted from the
[book](http://www.computervisionmodels.com/) "Computer Vision: Models, Learning, and
Inference" authored by Simon J. D. Prince.


## Navie Particle Filter

After providing an overview of our objective, let's delve into the specific details of
our task. To begin, it's crucial to establish the definition of our "model" or
"template" within this context. The "model" or "template" represents the object that we
aim to track. This entity could manifest as a patch in an image, a contour, or any other
descriptive representation of the object under consideration.

For the first task, we need to track a patch taken from the first frame of the video as
shown below, which is Mitt Romney's face. Our goal is to track this face throughout the
time. Thus we can define a `Template` which holds the information of the template as
follow:

```python
class Template:
    def __init__(self, img, x, y, w, h) -> None:
        self.x = int(x)
        self.y = int(y)
        self.w = int(w)
        self.h = int(h)
        self.model = img[self.y : self.y + self.h, self.x : self.x + self.w, ...]
```

Once we've established our template, it naturally leads to the design of the system's
state, which comprises the following four components: `(x, y, w, h)`. These components
represent the x-axis coordinate `x`, the y-axis coordinate `y`, the width of the window
`w`, and the height of the window `h`.


In [ ]:
first_frame = president_video[0, ...]  # NOTE: uint8. 0-255.
print("first_frame: ", first_frame.shape)
x, y, w, h = 320, 175, 103, 129
template = Template(first_frame, x, y, w, h)
print(
    "template.model:",
    "shape:", template.model.shape,
    "type:", template.model.dtype,
    "min:", np.min(template.model),
    "max:", np.max(template.model),
)

fig, ax = plt.subplots(figsize=(3, 2))  # Adjust the figure size as needed
plt.axis("off")
ax.imshow(template.model)
plt.show()


### Dynamic Model

Now that we have defined our system's state, we can proceed to define our dynamic
system. In this straightforward particle filter, we opt for the simplest form of motion,
known as Brownian motion:

$$
x_{k} = x_{t-1} + W_{k-1}
$$

The process noise represented by $W_{k-1}$ introduces variability in our dynamic model.
It's important to note that the range and magnitude of each dimension in the state and
the associated process noise can differ. For instance, when considering `x` and `y`,
these dimensions should remain within the boundaries of the image's dimensions.
Consequently, the standard deviation of their process noise should typically fall within
the range of 10 to 50. On the other hand, `w` and `h` should fall within an anticipated
range, typically around 10 to 25 pixels added or subtracted from the original patch
size. This choice is made based on practical considerations, as we wouldn't expect an
object, such as Mitt Romney's face, to occupy the entire image. Hence, the standard
deviation of the process noise of `w` and `h` should be only few pixels.

The choice of using Brownian motion as the dynamic model is grounded in the absence of
external control inputs to govern the object being tracked. In such scenarios, we can
assume that the object's movement is inherently random. The implementation is
straightforward, as outlined below. Essentially, during the prediction step, we
introduce Gaussian noise to the state vector. You might wonder about the magnitude of
noise added to the state. The magnitude varies depending on the specific state
dimension. For instance, if we have prior knowledge that the object in the image
typically moves only a few pixels in each time step, an appropriate value might range
from a sub ten to tens of pixels. It's important to note that in our task, we must
constrain the boundaries of the state, as it wouldn't be logical for image coordinates,
for example, to be negative.


In [ ]:
class DynamicModel:
    def __init__(self, std: List) -> None:
        self.std = std
        self.num_states = len(std)

    def predict(self, particles: np.ndarray, state_boundry: List[Tuple]):
        for i in range(self.num_states):
            # Brownian motion.
            particles[:, i] += np.random.normal(
                loc=0, scale=self.std[i], size=particles[:, i].shape
            )  # Modify particles in-place

            lower_bound, upper_bound = state_boundry[i]

            # Constrain state within boundry.
            np.clip(particles[:, i], lower_bound, upper_bound, out=particles[:, i])

### Measurement Model

now let's talk about measurement model. the measurement model is a bit more complicate
than dynamic model because observation data is all we have to correct the state.

there are many things needed to be taken care of:

-   measurement function: the measure function is:

$$
p(z|x) = e^{\frac{-distance}{2*\sigma}}
$$

the distance here is always positive, thus the expression $e^{-x}$ is always between 0
and 1, which can be quite useful and handful. the $\sigma$ is the standard deviation we
want to credit our observation data. The bigger std, the more uncertain we are about the
observation data. the smaller std, we can put more trust on the observation data. now,
what's tricky is that can we use a universal sigma for different kinds of distance
functions. I've tried three distance functions here, mean squared error, chi squared and
cosine similarity. I don't know why consine similariyt performs kind of wonky here,
since I think cosine similarity can also measure the similarity of two image patches.
mse and chi-squared both work. to my surprice, mse is actually performing better than
chi-squared and it's more computation efficient. the weight is 0 if their size doesn't
match or they are very different. The weight is 1 if they are exactly the same.

-   Get windows corresponding to each particle. For each particle, it has 4 states, x
    coord., y coord. width and height. We want to use (x, y, w, h) to crop a window from
    the given frame. two things need to be handle here. One, even though we have
    constrained states minimum and maximum value in the resampling stage, we still can
    not guarantee that (x, y, w, h) is a valid window. If we want to guarantee that (x,
    y, w, h) must be a valid window, we must also constained the relationship between x
    and w, y and h. However, it's not necessary. because in the later step, we can just
    reset the weight of the particle to 0 if the window is invalid.

    the second thing to notice is that the cropped window has the different size as the
    template, so we must resize the window to match template's size.

    the last thing is normalizing the weight so they sum to 1.

-   update_template: when tracking an target, the target appearance could change over
    time, thus we must also update our template over time. updating the template can be
    achieve with the eqaution:

    $$
    template = \alpha * state + (1 - \alpha) * template
    $$

    updating template is the process of fusion latest state and history states. now we
    don't want to always update the latest state because the latest state may be broken.
    so we must be cautious.


In [ ]:
class MeasurementModel:
    def __init__(self, std, template, distance_func, alpha=0.0) -> None:
        self.std = std
        self.template = template
        self.distance_func = distance_func
        self.alpha = alpha

    def measure(self, particles, frame):
        num_particles = particles.shape[0]
        # Get windows corresponding to each particle
        # Fix the size of template and resize each particle to match template
        template_width, template_height = self.template.w, self.template.h
        artificial_measurements = []
        for i in range(num_particles):
            x = particles[i, 0]  # float
            y = particles[i, 1]  # float
            w = particles[i, 2]  # float
            h = particles[i, 3]  # float
            start_y = int(y - h / 2)
            start_x = int(x - w / 2)
            end_y = int(start_y + h)
            end_x = int(start_x + w)
            # print(
            #     f"x: {x}, y: {y}, w: {w}, h: {h}, start_y: {start_y}, start_x: {start_x}, end_y: {end_y}, end_x: {end_x}"
            # )
            temp = frame[start_y:end_y, start_x:end_x, :]
            if temp.size == 0:
                # it's ok to still add it because the weight is reset as 0 in the latter process.
                artificial_measurements.append(temp)
            else:
                resized_image = cv2.resize(temp, (template_width, template_height))
                artificial_measurements.append(resized_image)

        # NOTE: Add measurement noise? window += np.random.normal(loc=0, scale=self.std, size=window.shape).astype(np.uint8)
        # Compute importance weight: Measuring similarity of each window to the template
        weights = []  # FIXME: Use array broadcasting
        for m in artificial_measurements:
            weights.append(self.measure_function(m, self.template.model))
        weights = np.array(weights)

        # NOTE: WE may want to clip weight here. eg, weight[weight<1e-3] = 0
        # so that particles with very small weight can have zero chances.
        # weights *= 1000
        # weights[weights < 1e-3] = 0
        # Normalize the weights
        weights /= np.sum(weights)
        return weights

    def measure_function(self, window, template):
        if window.shape != template.shape:
            # NOTE: This statement must be here. It can not be put into distance function.
            return 0
        dist = self.distance_func(window, template)
        weight = np.exp(-dist / (2 * self.std**2))

        return weight

    def update_template(self, frame, state):
        x = state[0]
        y = state[1]
        w = state[2]
        h = state[3]
        start_x = int(x - w / 2)
        start_y = int(y - h / 2)
        end_x = int(start_x + w)
        end_y = int(start_y + h)
        best_model = frame[start_y:end_y, start_x:end_x, ...]

        # resize current to best model bcs the best model shrinks.
        resized_template = cv2.resize(self.template.model, (w, h))
        # assert resized_template.shape == best_model.shape, f"x: {x}, y: {y}, w: {w}, h: {h}, resized_template: {resized_template.shape}, best_model: {best_model.shape}"

        if resized_template.shape != best_model.shape:
            return

        self.template.model = (
            self.alpha * best_model + (1 - self.alpha) * resized_template
        )
        # self.template.model = self.template.model.astype(np.uint8)

        # update x, y, w, h
        self.template.x = x
        self.template.y = y
        self.template.w = w
        self.template.h = h


def mean_squared_error(window, template):
    mse = np.sum(np.subtract(window, template, dtype=np.float64) ** 2)
    mse /= float(window.shape[0] * window.shape[1])
    return mse


def cosine_similarity(window, template):
    gray_window, gray_template = color.rgb2gray(window), color.rgb2gray(template)
    gray_window *= 255
    gray_template *= 255
    dividend = np.sum(np.multiply(gray_window, gray_template))
    divisor = np.multiply(
        np.sqrt(np.sum(gray_window**2)), np.sqrt(np.sum(gray_template**2))
    )
    assert np.all(divisor != 0), f"divisor has 0 in it. {divisor}"
    tmp = (np.divide(dividend, divisor) + 1) / 10000
    assert tmp > 0
    return tmp
    # dividend = np.sum(np.multiply(window, template))
    # divisor = np.multiply(np.sqrt(np.sum(window**2)), np.sqrt(np.sum(template**2)))
    # assert np.all(divisor != 0), f"divisor has 0 in it. {divisor}"
    # return 0.5 * (np.divide(dividend, divisor) + 1)


def chi_squared(window, template, num_bins=8):
    # Compute histograms for each channel
    hist1 = []
    hist2 = []
    for channel in range(window.shape[2]):
        hist1_channel, _ = np.histogram(
            window[:, :, channel], bins=num_bins, range=(0, 256)
        )
        hist2_channel, _ = np.histogram(
            template[:, :, channel], bins=num_bins, range=(0, 256)
        )

        # Normalize the histograms
        # hist1_channel = hist1_channel.astype(np.float64)
        # hist2_channel = hist2_channel.astype(np.float64)
        # hist1_channel /= hist1_channel.sum() + 1e-10
        # hist2_channel /= hist2_channel.sum() + 1e-10

        hist1.append(hist1_channel)
        hist2.append(hist2_channel)

    hist1 = np.array(hist1)
    hist2 = np.array(hist2)

    # Compute the Chi-Squared distance
    chi_squared_distance = np.sum(((hist1 - hist2) ** 2) / (hist1 + hist2 + 1e-10))
    return chi_squared_distance

### Naive Particle Filter

talk about update template, reset,

**num_states**: the number of states in the naive particle filter
**state_boundry**: for each state, it has a boundry.
**num_particles**: the number of particles in the navie particle filter.
**consensus**: it is from 0 to 1 used to represent a proportion of particles with larget
weights which will be used to determine some matter. It uses minor meritocraciy. only
top-tier weights have a say. if `consensus=0.05` and `num_particles=500`, then only
`0.05*500=25` particles with highest weights will be selected.

**upper_thres**: once elite particles have been selected, their sum must be higher than
a certain value 

In [ ]:
class NaiveParticleFilter:
    def __init__(
        self,
        num_states: int,
        state_boundry: List[Tuple],
        dynamic_model: DynamicModel,
        measure_model: MeasurementModel,
        num_particles=100,
        consensus=0.05,  # percentage
        upper_thres=0.3,  # percentage
        lower_thres=0.1,  # percentage
    ):
        self.num_states = num_states
        self.state_boundry = state_boundry
        self.num_particles = num_particles
        self.dynamic_model = dynamic_model
        self.measure_model = measure_model
        # Democracy here.
        self.num_consensus_particles = int(self.num_particles * consensus)
        self.upper_thres = upper_thres
        self.lower_thres = lower_thres

        self.particles = np.zeros((num_particles, num_states))  # (N, 4): x, y, w, h
        self.weights = np.ones(self.num_particles) / self.num_particles  # (N, ): weight

        self.reset_particles()
        self.state = self.particles[69, :] # Initialize state randomly

    def reset_particles(self):
        for i in range(self.num_states):
            lower_bound, upper_bound = self.state_boundry[i]
            self.particles[:, i] = np.random.uniform(
                lower_bound, upper_bound, self.num_particles
            )
        # Rest weights. each particle has the same weight.
        self.weights = np.ones(self.num_particles) / self.num_particles

    def update(self, frame):
        sorted_indices = np.argsort(self.weights)
        largest_indices = sorted_indices[-self.num_consensus_particles :]
        # if (
        #     np.sum(self.weights[largest_indices])
        #     >  self.lower_thres
        # ):
        self.resample_particles()
        
        self.dynamic_model.predict(self.particles, self.state_boundry)

        self.weights = self.measure_model.measure(self.particles, frame)

        # Add very certain here

        # NOTE: MAYBE not always update states. think about occlusion.
        self.update_states()

        if np.sum(self.weights[largest_indices]) > self.upper_thres:
            self.measure_model.update_template(frame, self.state)

        # if np.sum(self.weights[largest_indices]) < self.lower_thres:
        #     # reset particles is kind of stupid. it destroys all prior info.
        #     self.reset_particles()

    def resample_particles(self):
        # sample new particle indices using the distribution of the weights
        j = np.random.choice(
            np.arange(self.num_particles),
            self.num_particles,
            replace=True,
            p=self.weights,
        )

        # get a random control input from a normal distribution
        # sample the particles using the distribution of the weights
        self.particles = np.array(self.particles[j])
        assert self.particles.shape[0] == self.num_particles

        # clip particles in case the window goes out of the image limits
        for i in range(self.num_states):
            lower_bound, upper_bound = self.state_boundry[i]
            np.clip(
                self.particles[:, i], lower_bound, upper_bound, out=self.particles[:, i]
            )

        # Rest weights. each particle has the same weight.
        self.weights = np.ones(self.num_particles) / self.num_particles

    def update_states(self):
        sorted_indices = np.argsort(self.weights)
        largest_indices = sorted_indices[-self.num_consensus_particles :]
        s = np.sum(self.particles[largest_indices, :], axis=0)
        average = s / len(largest_indices)
        self.state = average.astype(int)


### Navie example 
frame = (720, 1280, 3)
in this example, 

In [ ]:
parameter = [(mean_squared_error, 16, 16, 0.01, 2), (chi_squared, 16, 16, 0.01, 2)]

tracker = NaiveParticleFilter(
    num_states=4,
    state_boundry=[
        (0, first_frame.shape[1] - 1),
        (0, first_frame.shape[0] - 1),
        (100, 105),
        (125, 135),
    ],
    dynamic_model=DynamicModel(std=(16, 16, 0.5, 0.5)),
    measure_model=MeasurementModel(
        std=16,
        template=template,
        distance_func=partial(mean_squared_error),
        # distance_func=partial(chi_squared, num_bins=12),
        # distance_func=partial(cosine_similarity),
        alpha=0.01,
    ),
    num_particles=128,
    consensus=0.20,  # percentage
    upper_thres=0.2,  # percentage
    lower_thres=0.1,  # percentage
)

for i in range(1, president_video.shape[0]):
    start_time = time.time()

    frame = president_video[i, ...]

    tracker.update(frame)

    frame = visualize_particle_filter(
        frame, tracker.particles, tracker.state, tracker.measure_model.template
    )

    delay = int(25 - (time.time() - start_time))
    if cv2.waitKey(delay) & 0xFF == ord("q"):
        break
    cv2.imshow("pres_debate", frame[:, :, ::-1])  # Display the resulting frame
    # ax.imshow(frame)
    # display(fig)
    # clear_output(wait=True)
    # time.sleep(1)

cv2.destroyAllWindows()

In [ ]:
# class DynamicModel:
#     def __init__(self, std) -> None:
#         self.std = std

#     def predict(self, particles):
#         # The simplest dyanmic model is brownian motion.
#         particles += np.random.normal(loc=0, scale=self.std, size=particles.shape)


# class MeasurementModel:
#     def __init__(self, std, template, distance_func, alpha=0.0, update_thres=2) -> None:
#         self.std = std
#         self.template = template
#         self.distance_func = distance_func
#         self.alpha = alpha
#         self.update_thres = update_thres

#     def measure(self, particles, frame):
#         num_particles = particles.shape[0]
#         # Get windows corresponding to each particle
#         win_height, win_width = self.template.model.shape[:2]
#         start_x = (particles[:, 0] - win_width / 2).astype(int)
#         start_y = (particles[:, 1] - win_height / 2).astype(int)
#         end_x = start_x + win_width
#         end_y = start_y + win_height

#         artificial_measurements = []
#         for i in range(num_particles):
#             window = frame[start_y[i] : end_y[i], start_x[i] : end_x[i]]
#             # NOTE: Add measurement noise? window += np.random.normal(loc=0, scale=self.std, size=window.shape).astype(np.uint8)
#             artificial_measurements.append(window)

#         # Compute importance weight: Measuring similarity of each window to the template
#         weights = []
#         for m in artificial_measurements:
#             weights.append(self.measure_function(m, self.template.model))
#         weights = np.array(weights)

#         # Normalize the weights
#         weights /= np.sum(weights)

#         # Only update template if very certain.
#         best_index = np.argmax(weights)
#         if weights[best_index] > self.update_thres * 1 / num_particles:
#             self.update_template(frame, particles[best_index, :])

#         return weights

#     def measure_function(self, window, template):
#         if window.shape != template.shape:
#             # NOTE: This statement must be here. It can not be put into distance function.
#             return 0
#         dist = self.distance_func(window, template)
#         return np.exp(-dist / (2 * self.std**2))

#     def update_template(self, frame, states):
#         w = self.template.w
#         h = self.template.h
#         x = int(states[0] - w / 2)
#         y = int(states[1] - h / 2)
#         best_model = frame[y : y + h, x : x + w]
#         if best_model.shape == self.template.model.shape:
#             self.template.model = (
#                 self.alpha * best_model + (1 - self.alpha) * self.template.model
#             )
#             # self.template.model = self.template.model.astype(np.uint8)


# def mean_squared_error(window, template):
#     mse = np.sum(np.subtract(window, template, dtype=np.float64) ** 2)
#     mse /= float(window.shape[0] * window.shape[1])
#     return mse


# def cosine_similarity(window, template):
#     gray_window, gray_template = color.rgb2gray(window), color.rgb2gray(template)
#     gray_window *= 255
#     gray_template *= 255
#     dividend = np.sum(np.multiply(gray_window, gray_template))
#     divisor = np.multiply(
#         np.sqrt(np.sum(gray_window**2)), np.sqrt(np.sum(gray_template**2))
#     )
#     assert np.all(divisor != 0), f"divisor has 0 in it. {divisor}"
#     tmp = (np.divide(dividend, divisor)+1) / 10000
#     assert tmp > 0
#     return tmp
#     # dividend = np.sum(np.multiply(window, template))
#     # divisor = np.multiply(np.sqrt(np.sum(window**2)), np.sqrt(np.sum(template**2)))
#     # assert np.all(divisor != 0), f"divisor has 0 in it. {divisor}"
#     # return 0.5 * (np.divide(dividend, divisor) + 1)


# def chi_squared(window, template, num_bins=8):
#     # Compute histograms for each channel
#     hist1 = []
#     hist2 = []
#     for channel in range(window.shape[2]):
#         hist1_channel, _ = np.histogram(
#             window[:, :, channel], bins=num_bins, range=(0, 256)
#         )
#         hist2_channel, _ = np.histogram(
#             template[:, :, channel], bins=num_bins, range=(0, 256)
#         )

#         # Normalize the histograms
#         # hist1_channel = hist1_channel.astype(np.float64)
#         # hist2_channel = hist2_channel.astype(np.float64)
#         # hist1_channel /= hist1_channel.sum() + 1e-10
#         # hist2_channel /= hist2_channel.sum() + 1e-10

#         hist1.append(hist1_channel)
#         hist2.append(hist2_channel)

#     hist1 = np.array(hist1)
#     hist2 = np.array(hist2)

#     # Compute the Chi-Squared distance
#     chi_squared_distance = np.sum(((hist1 - hist2) ** 2) / (hist1 + hist2 + 1e-10))
#     return chi_squared_distance
    

# class NaiveParticleFilter:
#     def __init__(
#         self,
#         search_height,
#         search_width,
#         dynamic_model: DynamicModel,
#         measure_model: MeasurementModel,
#         num_particles=100,
#         consensus=20, # percentage
#     ):
#         self.search_height = search_height
#         self.search_width = search_width
#         self.num_particles = num_particles
#         self.dynamic_model = dynamic_model
#         self.measure_model = measure_model
#         self.num_consensus_particles = int(self.num_particles / consensus)

#         particles = []
#         for x, y in zip(
#             np.random.uniform(0, self.search_width, self.num_particles),
#             np.random.uniform(0, self.search_height, self.num_particles),
#         ):
#             particles.append((x, y))

#         self.particles = np.array(particles)  # (N, 2): x, y
#         self.weights = np.ones(self.num_particles) / self.num_particles  # (N, ): weight
#         self.states = self.particles[0, :]

#     def update(self, frame):
#         self.resample_particles()
#         self.dynamic_model.predict(self.particles)
#         self.weights = self.measure_model.measure(self.particles, frame)
#         self.update_states()

#     def resample_particles(self):
#         sw, sh = self.search_width, self.search_height

#         # sample new particle indices using the distribution of the weights
#         j = np.random.choice(
#             np.arange(self.num_particles),
#             self.num_particles,
#             replace=True,
#             p=self.weights,
#         )
#         # get a random control input from a normal distribution
#         # sample the particles using the distribution of the weights
#         self.particles = np.array(self.particles[j])
#         assert self.particles.shape[0] == self.num_particles
#         # clip particles in case the window goes out of the image limits
#         self.particles[:, 0] = np.clip(self.particles[:, 0], 0, sw - 1)
#         self.particles[:, 1] = np.clip(self.particles[:, 1], 0, sh - 1)

#         # Rest weights. each particle has the same weight.
#         self.weights = np.ones(self.num_particles) / self.num_particles

#     def update_states(self):
#         best_index = np.argmax(self.weights)
#         self.states = self.particles[best_index, :]

In [ ]:
# patch = president_video[0, ...]  # NOTE: uint8. 0-255.
# x, y, w, h = 320.8751, 175.1776, 103.5404, 129.0504
# template = Template(patch, x, y, w, h)

# fig, ax = plt.subplots(figsize=(6, 4))  # Adjust the figure size as needed
# plt.axis("off")
# ax.imshow(template.model)
# plt.show()
# print(
#     "template.model: ",
#     template.model.shape,
#     template.model.dtype,
#     np.min(template.model),
#     np.max(template.model),
# )

# parameter = [(mean_squared_error, 16, 16, 0.01, 2), (chi_squared, 16, 16, 0.01, 2)]

# tracker = NaiveParticleFilter(
#     patch.shape[1],  # can be replace with template
#     patch.shape[0],
#     DynamicModel(std=16),
#     MeasurementModel(
#         std=16,
#         template=template,
#         distance_func=mean_squared_error,
#         # distance_func=partial(chi_squared, num_bins=12),
#         # distance_func=cosine_similarity,
#         alpha=0.2,
#         update_thres=2,
#     ),
#     num_particles=100,
# )

# for i in range(1, president_video.shape[0]):
#     start_time = time.time()

#     frame = president_video[i, ...]

#     tracker.update(frame)

#     frame = visualize_particle_filter(
#         frame, tracker.particles, tracker.states, tracker.measure_model.template
#     )

#     delay = int(25 - (time.time() - start_time))
#     if cv2.waitKey(delay) & 0xFF == ord("q"):
#         break
#     cv2.imshow("pres_debate", frame[:, :, ::-1])  # Display the resulting frame
#     # ax.imshow(frame)
#     # display(fig)
#     # clear_output(wait=True)
#     # time.sleep(1)

# cv2.destroyAllWindows()

In [ ]:
# output_file = "./images/romney.mp4"

# video_writer = imageio.get_writer(output_file, fps=30)  # You can adjust the FPS as needed
# for frame in final_output:
#     video_writer.append_data(frame)

# # Close the video writer
# video_writer.close()

## More realworld example

frame is (360, 480, 3)





### dynamic model

### measurement model

### Read world particle filter


In [ ]:
# patch = blonde_video[0, ...]
# x, y, w, h = 211.0000, 36.0000, 100.0000, 293.0000
# template = Template(patch, x, y, w, h)

# fig, ax = plt.subplots(figsize=(6, 4))  # Adjust the figure size as needed
# plt.axis("off")
# ax.imshow(template.model)
# plt.show()
# print(
#     "template.model: ",
#     template.model.shape,
#     template.model.dtype,
#     np.min(template.model),
#     np.max(template.model),
# )


# tracker = NaiveParticleFilter(
#     patch.shape[1],
#     patch.shape[0],
#     DynamicModel(std=10),
#     MeasurementModel(
#         std=10,
#         template=template,
#         measure_func=partial(mean_squared_error, mse_std=100),
#         # measure_func=partial(chi_squared, num_bins=16),
#         # measure_func=partial(cosine_similarity),
#         alpha=0.0075,
#     ),
#     num_particles=100,
# )

# for i in range(1, blonde_video.shape[0]):
#     start_time = time.time()

#     frame = blonde_video[i, ...]

#     tracker.update(frame)

#     frame = visualize_particle_filter(
#         frame, tracker.particles, tracker.states, tracker.measure_model.template
#     )

#     delay = int(25 - (time.time() - start_time))
#     if cv2.waitKey(delay) & 0xFF == ord("q"):
#         break
#     cv2.imshow("pres_debate", frame[:, :, ::-1])  # Display the resulting frame
#     # ax.imshow(frame)
#     # display(fig)q
#     # clear_output(wait=True)
#     # time.sleep(1)

# cv2.destroyAllWindows()

In [ ]:
first_frame = blonde_video[0, ...]  # NOTE: uint8. 0-255.
print("first_frame: ", first_frame.shape)
x, y, w, h = 211, 36, 100, 293
template = Template(first_frame, x, y, w, h)

fig, ax = plt.subplots(figsize=(3, 2))  # Adjust the figure size as needed
plt.axis("off")
ax.imshow(template.model)
plt.show()
print(
    "template.model: ",
    template.model.shape,
    template.model.dtype,
    np.min(template.model),
    np.max(template.model),
)

parameter = [(mean_squared_error, 16, 16, 0.01, 2), (chi_squared, 16, 16, 0.01, 2)]

tracker = NaiveParticleFilter(
    num_states=4,
    state_boundry=[
        (0, first_frame.shape[1] - 1),
        (0, first_frame.shape[0] - 1),
        (10, 100),
        (20, 290),
    ],
    dynamic_model=DynamicModel(std=(16, 16, 1, 1)),
    measure_model=MeasurementModel(
        std=16,
        template=template,
        distance_func=mean_squared_error,
        # distance_func=partial(chi_squared, num_bins=12),
        # distance_func=cosine_similarity,
        alpha=0.01,
    ),
    num_particles=2048,
    consensus=0.2,  # percentage
    upper_thres=0.2,  # percentage
    lower_thres=0.1,  # percentage
)

for i in range(1, blonde_video.shape[0]):
    start_time = time.time()

    frame = blonde_video[i, ...]

    tracker.update(frame)

    frame = visualize_particle_filter(
        frame, tracker.particles, tracker.state, tracker.measure_model.template
    )

    delay = int(25 - (time.time() - start_time))
    if cv2.waitKey(delay) & 0xFF == ord("q"):
        break
    cv2.imshow("blonde", frame[:, :, ::-1])  # Display the resulting frame
    # ax.imshow(frame)
    # display(fig)
    # clear_output(wait=True)
    # time.sleep(1)

cv2.destroyAllWindows()